In [1]:
import os, time, sys
from pathlib import Path

SESSION_NAME = "DS5_Bode_Museum_Filter3"

ROOT = Path().absolute().parent
RAW_INPUT_PATH = Path(f"{ROOT}/data/1_raw_input")
INPUT_PATH = Path(f"{ROOT}/data/2_input")
RESULTS_PATH = Path(f"{ROOT}/data/3_results")

SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")

!mkdir -p {RAW_INPUT_PATH}
!mkdir -p {INPUT_PATH}
!mkdir -p {SESSION_PATH}

print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID DS5_Bode_Museum_Filter3 under /mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results


In [ ]:
##### Data Preparation ( raw input -> input folder)
%cd {ROOT}

raw_input_files = [x for x in RAW_INPUT_PATH.glob('**/*') if x.is_file()]
print(f"found {len(raw_input_files)} raw input files")
_f = "/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4" #str(raw_input_files[0])
print(_f)
DOWNSAMPLING_RATE = 2

#!bash scripts/third_party/neuralangelo/run_ffmpeg.sh {{SESSION_NAME}} {{_f}} 2
! ffmpeg -i {_f} -vf "select=not(mod(n\,{DOWNSAMPLING_RATE}))" -vsync vfr -q:v 2 {INPUT_PATH}/%06d.jpg

# Reconstruction (please read)
You have two options to continue here:
1. Reconstruction with Colmap wrapper in Python (with pycolmap) *
2. Reconstruction with Colmap (**PREFERRED**) **

1* pycolmap only supports sparse reconstruction, unless you build colmap from source with CUDA support

2** Colmap needs to be build and installed on your environment. If it is, you should use it. Dense reconstruction requires CUDA support on the machine.

## Reconstruction with pycolmap here

In [ ]:
# SPARSE RECONSTRUCTION: Step 1/3
import pycolmap

COLMAP_SFM_PATH = Path(f"{SESSION_PATH}/sparse")
COLMAP_MVS_PATH = Path(f"{SESSION_PATH}/dense")
COLMAP_DB_PATH = Path(f"{SESSION_PATH}/database.db")

!mkdir -p {COLMAP_MVS_PATH}
!mkdir -p {COLMAP_SFM_PATH}

pycolmap.extract_features(COLMAP_DB_PATH, INPUT_PATH)

In [ ]:
# SPARSE RECONSTRUCTION: Step 2/3
pycolmap.match_exhaustive(COLMAP_DB_PATH)

In [ ]:
# SPARSE RECONSTRUCTION: Step 3/3
maps = pycolmap.incremental_mapping(COLMAP_DB_PATH, INPUT_PATH, COLMAP_SFM_PATH)
maps[0].write(COLMAP_SFM_PATH)

In [ ]:
# DENSE RECONSTRUCTION: Step 1/3
pycolmap.undistort_images(COLMAP_MVS_PATH, COLMAP_SFM_PATH, INPUT_PATH)

In [ ]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.patch_match_stereo(COLMAP_MVS_PATH)  # requires compilation with CUDA

In [ ]:
# DENSE RECONSTRUCTION: Step 3/3
pycolmap.stereo_fusion(COLMAP_MVS_PATH / "dense.ply", COLMAP_MVS_PATH)

## Reconstruction with COLMAP here

In [2]:
# Sparse Reconstruction (Step 1)
! bash run_colmap_sparse.sh "{SESSION_PATH}" "{INPUT_PATH}"

I0213 21:48:50.254913  1027 misc.cc:198] 
Feature extraction
I0213 21:48:50.816954  1061 feature_extraction.cc:254] Processed file [1/27]
I0213 21:48:50.816999  1061 feature_extraction.cc:257]   Name:            IMG_0712.png
I0213 21:48:50.817006  1061 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0213 21:48:50.817009  1061 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I0213 21:48:50.817013  1061 feature_extraction.cc:289]   Focal Length:    1965.60px
I0213 21:48:50.817020  1061 feature_extraction.cc:302]   Features:        11141
I0213 21:48:51.623909  1061 feature_extraction.cc:254] Processed file [2/27]
I0213 21:48:51.623939  1061 feature_extraction.cc:257]   Name:            IMG_0713.png
I0213 21:48:51.623943  1061 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0213 21:48:51.623946  1061 feature_extraction.cc:286]   Camera:          #2 - SIMPLE_RADIAL
I0213 21:48:51.623950  1061 feature_extraction.cc:289]   Focal Length:    1965.60px
I0213

In [ ]:
# Dense Reconstruction IF NEEDED (Step 2)
! bash run_colmap_dense.sh "{SESSION_PATH}"

## Generate Transforms File
Generate this file if you want to have camera, points etc. in a self contained file which you can then use to display everything in a 3D engine like Blender (see my scripts `colmap_pc_importer_ui.py` and `colmap_pc_importer.py` which u can load into Blender)

In [3]:
import sys

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

print(sys.path)

generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json_advanced.py"
! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor" --image_dir "{INPUT_PATH}"

#generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json.py"
#! python {generate_transforms_script} --data_dir "{SESSION_PATH}" --scene_type "outdoor"

['/mnt/d/dev/python/historical-photo-sfm-pipeline/scripts', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python39.zip', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/lib-dynload', '', '/root/miniconda3/envs/historical-photo-sfm-pipeline/lib/python3.9/site-packages', '/mnt/d/dev/python/historical-photo-sfm-pipeline']
/mnt/d/dev/python/historical-photo-sfm-pipeline
Fraction of images looking at the center: 0.37.
Fraction of images positioned around the center: 0.67.
Valid fraction of concentric images: 0.30.
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7F9AE60925B0>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7F9AE127CB20>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7F9AE1206520>
No EXIF data found for image <PIL.PngImagePlugin.P

In [ ]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()

        if exif_data is not None:
            # Decode the EXIF data
            decoded_exif = {TAGS[key]: exif_data[key] for key in exif_data.keys() if key in TAGS and isinstance(exif_data[key], (int, str, bytes))}
            return decoded_exif
        else:
            print("No EXIF data found.")
            return None

    except Exception as e:
        print(f"Error reading EXIF data: {e}")
        return None

# Example usage
image_path = '/mnt/d/dev/python/historical-photo-sfm-pipeline/data/2_input/IMG_0555.jpeg'
exif_data = get_exif_data(image_path)

if exif_data:
    print("EXIF Metadata:")
    for key, value in exif_data.items():
        print(f"{key}: {value}")

In [ ]:
'''
camera = pycolmap.Camera(
    model=camera_model_name_or_id,
    width=width,
    height=height,
    params=params,
)

import pycolmap
reconstruction = pycolmap.Reconstruction("path/to/reconstruction/dir")
print(reconstruction.summary())

for image_id, image in reconstruction.images.items():
    print(image_id, image)

for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

reconstruction.write("path/to/reconstruction/dir/")
'''